<a href="https://colab.research.google.com/github/Onethybeing/pytorch_implementation/blob/main/notebookce902e5183.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
path = kagglehub.dataset_download('arshid/iris-flower-dataset')

print('Data source import complete.')


Data source import complete.


In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/iris-flower-dataset/IRIS.csv


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.metrics import accuracy_score

In [8]:
import os

# Look at the downloaded files
print(os.listdir(path))

# Assuming the file is iris.csv or similar
df = pd.read_csv(os.path.join(path, "IRIS.csv"))
print(df.head())


['IRIS.csv']
   sepal_length  sepal_width  petal_length  petal_width      species
0           5.1          3.5           1.4          0.2  Iris-setosa
1           4.9          3.0           1.4          0.2  Iris-setosa
2           4.7          3.2           1.3          0.2  Iris-setosa
3           4.6          3.1           1.5          0.2  Iris-setosa
4           5.0          3.6           1.4          0.2  Iris-setosa


In [23]:
df["species"].nunique()

3

In [9]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [10]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [11]:
label_encoder ={}
cat_columns =['species']
for cat  in cat_columns:
    label_encoder[cat] = LabelEncoder()
    train_df[cat] = label_encoder[cat].fit_transform(train_df[cat])
    test_df[cat] = label_encoder[cat].transform(test_df[cat])

In [12]:
x_train = train_df.drop('species',axis=1).values
y_train = train_df['species'].values
x_test = test_df.drop('species',axis=1).values
y_test = test_df['species'].values

In [14]:
x_train.shape

(120, 4)

In [15]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [36]:
class IrisClassifier(nn.Module):
  def __init__(self,input_dim,hidden_dim,_output_dim):
    super(IrisClassifier,self).__init__()
    self.network = nn.Sequential( # Fixed: Sequential was not assigned to a variable
        nn.Linear(input_dim,hidden_dim),
        nn.ReLU(),
        nn.Linear(hidden_dim,hidden_dim),
        nn.ReLU(),
        nn.Linear(hidden_dim,_output_dim),
    )
  def forward(self,x):
      return self.network(x)

In [37]:
x_train.shape

(120, 4)

In [55]:
x_train_tensor = torch.tensor(x_train,dtype= torch.float32)
x_test_tensor = torch.tensor(x_test,dtype= torch.float32)
y_train_tensor = torch.tensor(y_train,dtype= torch.long) # Changed to LongTensor
y_test_tensor = torch.tensor(y_test,dtype= torch.long) # Changed to LongTensor

In [56]:
input_dim = x_train.shape[1]
hidden_dim = 16
output_dim = 3

In [57]:
model = IrisClassifier(input_dim,hidden_dim,output_dim)

In [58]:
criteria = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.01)

In [59]:
epochs =500
for epoch in range(epochs): # Changed variable name to epoch to avoid conflict
  model.train()
  optimizer.zero_grad()
  predictions = model(x_train_tensor)
  loss = criteria(predictions,y_train_tensor)
  loss.backward()
  optimizer.step()
  if(epoch+1)%50 ==0: # Changed variable name to epoch
    print(f"Epoch[{epoch+1}/{epochs}],loss:{loss.item():.4f}") # Changed variable name to epoch and fixed f-string

Epoch[50/500],loss:0.0753
Epoch[100/500],loss:0.0385
Epoch[150/500],loss:0.0361
Epoch[200/500],loss:0.0349
Epoch[250/500],loss:0.0343
Epoch[300/500],loss:0.0340
Epoch[350/500],loss:0.0339
Epoch[400/500],loss:0.0338
Epoch[450/500],loss:0.0338
Epoch[500/500],loss:0.0338


In [64]:
model.eval()
with torch.no_grad():
  y_pred = model(x_test_tensor)
  y_pred_labels = torch.argmax(y_pred,dim=1)
  accuracy = (y_pred_labels == y_test_tensor).sum().item()/y_test_tensor.size(0)
  print(f"Test Accuracy:{accuracy:.2f}%")

Test Accuracy:1.00%


In [77]:
def predict_iris(sepal_length ,sepal_width,petal_length,petal_width):
  input_data = np.array([[sepal_length,sepal_width,petal_length,petal_width]])
  input_data = scaler.transform(input_data)
  input_tensor = torch.tensor(input_data,dtype=torch.float32)
  model.eval()
  with torch.no_grad():
    prediction = model(input_tensor)
    predicted_class = torch.argmax(prediction,dim=1).item()
  return label_encoder['species'].inverse_transform([predicted_class])[0]

In [79]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [81]:
predicted_class = predict_iris(5.1	,3.5	,1.4	,0.2)
print(f"Predicted Iris Species:{predicted_class}")

Predicted Iris Species:Iris-setosa
